# Statistics for LLM Development

This notebook covers the statistical concepts you need to understand how large language models work, why they are designed the way they are, and what is actually happening when they train. Each concept is introduced with theory first, then implemented in Python so you can see the numbers and build real intuition.

The concepts here are not an exhaustive statistics course. They are selected specifically because they appear directly in the mathematics of tokenization, attention mechanisms, loss functions, optimization, and model evaluation — the exact components you build when reading Raschka's book.

**Concepts covered:**

1. Probability Fundamentals
2. Probability Distributions
3. Expectation, Variance, and Standard Deviation
4. Information Theory: Entropy, Cross-Entropy, KL Divergence
5. Maximum Likelihood Estimation
6. Bayes' Theorem and Conditional Probability
7. The Normal Distribution and Why It Matters
8. Sampling and Temperature
9. Descriptive Statistics for Model Monitoring
10. Correlation and Covariance
11. Central Limit Theorem
12. Putting It All Together: A Language Model Probability Chain

---
## Setup

In [ ]:
!pip install --upgrade numpy scipy matplotlib --quiet

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 4)
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

print("NumPy  :", np.__version__)
print("SciPy  :", __import__('scipy').__version__)

---
## 1. Probability Fundamentals

### Theory

Probability is a number between 0 and 1 that quantifies how likely an event is to occur. P(A) = 0 means the event is impossible. P(A) = 1 means it is certain.

**Sample space** is the set of all possible outcomes. For a fair coin toss: {Heads, Tails}. For a six-sided die: {1, 2, 3, 4, 5, 6}.

**Key rules:**

The sum rule: P(A or B) = P(A) + P(B) - P(A and B)

The product rule: P(A and B) = P(A) * P(B|A), where P(B|A) is the probability of B given A has occurred.

If A and B are independent: P(A and B) = P(A) * P(B)

The complement rule: P(not A) = 1 - P(A)

**Where this appears in LLMs:** A language model is a probability distribution over sequences of tokens. When you ask GPT to complete a sentence, it computes P(next token | all previous tokens) for every token in the vocabulary and samples from that distribution. Every output is a probability calculation.

In [ ]:
vocab = ['the', 'cat', 'sat', 'on', 'mat', 'dog', 'ran', 'away']

raw_counts = np.array([120, 45, 30, 80, 25, 40, 20, 15])

probabilities = raw_counts / raw_counts.sum()

print("Token probabilities:")
for token, prob in zip(vocab, probabilities):
    print(f"  P('{token}') = {prob:.4f}")

print(f"\nSum of all probabilities: {probabilities.sum():.6f}")

p_the_or_cat = probabilities[0] + probabilities[1]
print(f"\nP('the' or 'cat') = {p_the_or_cat:.4f}")

p_not_the = 1 - probabilities[0]
print(f"P(not 'the')       = {p_not_the:.4f}")

This is exactly what a language model's output layer does. The final linear layer produces raw scores (logits) for every token in the vocabulary. A softmax function then converts those scores into probabilities that sum to 1. The model then either picks the highest probability token (greedy decoding) or samples from the distribution (temperature sampling).

The calculation `raw_counts / raw_counts.sum()` is conceptually identical to what softmax does — it normalizes a set of non-negative values to produce a valid probability distribution.

In [ ]:
p_the  = 0.32
p_cat_given_the = 0.15

p_the_then_cat = p_the * p_cat_given_the
print(f"P('the') = {p_the}")
print(f"P('cat' | 'the') = {p_cat_given_the}")
print(f"P('the', 'cat') = P('the') * P('cat'|'the') = {p_the_then_cat:.4f}")

sequence = ['the', 'cat', 'sat']
p_seq_given_prev = [p_the, p_cat_given_the, 0.40]

p_sequence = 1.0
for token, p in zip(sequence, p_seq_given_prev):
    p_sequence *= p

print(f"\nP(the cat sat) = {p_the} * {p_cat_given_the} * 0.40")
print(f"P(the cat sat) = {p_sequence:.6f}")
print(f"\nThis is the chain rule of probability — the foundation of language modeling.")

The chain rule of probability is the mathematical foundation of language modeling. A language model learns to estimate P(token_t | token_1, token_2, ..., token_{t-1}) — the probability of the next token given everything that came before. The probability of an entire sequence is the product of all these conditional probabilities. This is why sequences get exponentially less probable as they get longer, and why log probabilities are used in practice instead of raw probabilities — multiplying many small numbers causes numerical underflow.

---
## 2. Probability Distributions

### Theory

A probability distribution describes how probability is spread across all possible outcomes. For discrete outcomes (like token indices), this is a probability mass function. For continuous values (like weight initializations), this is a probability density function.

**Uniform distribution:** Every outcome is equally likely. A fair die follows a uniform distribution over {1, 2, 3, 4, 5, 6}.

**Categorical distribution:** A generalization of the coin flip to multiple outcomes with different probabilities. This is exactly what a language model produces — a categorical distribution over all tokens in the vocabulary.

**Normal (Gaussian) distribution:** The bell curve. Defined by mean (center) and variance (spread). The majority of values fall within two standard deviations of the mean.

**Uniform continuous distribution:** Values are equally likely across a continuous range [a, b].

**Where this appears in LLMs:** Weight initialization uses specific distributions (uniform or normal) chosen to prevent vanishing and exploding gradients. The output distribution is categorical. Noise added during training (dropout) follows a Bernoulli distribution.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

vocab_size = 10
uniform_probs = np.ones(vocab_size) / vocab_size
axes[0].bar(range(vocab_size), uniform_probs, color='steelblue', alpha=0.8)
axes[0].set_title('Uniform Distribution\n(equal probability — random model)')
axes[0].set_xlabel('Token index')
axes[0].set_ylabel('Probability')
axes[0].set_ylim(0, 0.5)

logits = np.array([0.5, -1.2, 2.3, 0.1, -0.8, 1.5, -0.3, 0.9, -1.1, 0.4])
exp_logits = np.exp(logits - logits.max())
softmax_probs = exp_logits / exp_logits.sum()
axes[1].bar(range(vocab_size), softmax_probs, color='coral', alpha=0.8)
axes[1].set_title('Categorical Distribution\n(softmax output — trained model)')
axes[1].set_xlabel('Token index')
axes[1].set_ylim(0, 0.5)

normal_samples = np.random.randn(5000)
axes[2].hist(normal_samples, bins=60, color='mediumseagreen', alpha=0.8, density=True)
x = np.linspace(-4, 4, 300)
axes[2].plot(x, stats.norm.pdf(x), 'k-', linewidth=2)
axes[2].set_title('Normal Distribution\n(weight initialization)')
axes[2].set_xlabel('Value')
axes[2].set_ylabel('Density')

plt.tight_layout()
plt.show()

print(f"Uniform distribution sums to: {uniform_probs.sum():.4f}")
print(f"Softmax distribution sums to: {softmax_probs.sum():.4f}")
print(f"Most likely token: index {softmax_probs.argmax()} with prob {softmax_probs.max():.4f}")

The left plot shows what an untrained model looks like — every token is equally likely, which is useless. The middle plot shows what a trained model produces — a peaked distribution where a few tokens have high probability. The right plot shows the normal distribution used to initialize weights. 

Why normal initialization? If weights start at zero, all neurons compute the same gradient and learn the same thing (symmetry problem). If weights are too large, gradients explode. If too small, gradients vanish. Normal initialization with small variance (std ≈ 0.02 for GPT-2) breaks symmetry while keeping values in a numerically stable range.

In [ ]:
n_in  = 768
n_out = 768

gpt2_init = np.random.normal(loc=0.0, scale=0.02, size=(n_in, n_out))

xavier_std = np.sqrt(2.0 / (n_in + n_out))
xavier_init = np.random.normal(loc=0.0, scale=xavier_std, size=(n_in, n_out))

he_std = np.sqrt(2.0 / n_in)
he_init = np.random.normal(loc=0.0, scale=he_std, size=(n_in, n_out))

print("Weight initialization strategies for a 768x768 layer:\n")
for name, weights in [("GPT-2 (std=0.02)", gpt2_init),
                      ("Xavier", xavier_init),
                      ("He", he_init)]:
    print(f"  {name:25s} | mean: {weights.mean():.5f} | std: {weights.std():.5f} "
          f"| range: [{weights.min():.3f}, {weights.max():.3f}]")

GPT-2 uses a fixed standard deviation of 0.02 for all weight matrices. Xavier initialization scales the standard deviation based on the number of input and output units — it is designed to keep the variance of activations roughly constant through layers when using tanh activations. He initialization uses only the input size and is designed for ReLU activations. When you read the GPT-2 paper or Raschka's implementation, you will see `torch.nn.init.normal_(weight, mean=0.0, std=0.02)` — now you know exactly what that is doing and why.

---
## 3. Expectation, Variance, and Standard Deviation

### Theory

**Expectation (Mean):** The expected value E[X] is the probability-weighted average of all possible values. For a discrete distribution: E[X] = sum of (x * P(X = x)) for all x. For a dataset of n values: E[X] = (1/n) * sum of all values.

**Variance:** Measures how spread out a distribution is. Var(X) = E[(X - E[X])^2]. It is the average squared deviation from the mean. Squaring ensures positive values and penalizes large deviations more than small ones.

**Standard deviation:** The square root of variance. SD = sqrt(Var(X)). It is in the same units as the original data, making it more interpretable.

**Bias-variance tradeoff:** In machine learning, a model with high bias underfits (makes strong, wrong assumptions). A model with high variance overfits (memorizes training data, fails on new data). Training is the process of finding the right balance.

**Where this appears in LLMs:** Layer normalization (used in every Transformer block) computes the mean and variance of activations and uses them to normalize. Gradient clipping uses the norm (related to variance) to stabilize training. Loss curves are monitored using running means and variances.

In [ ]:
data = np.array([2.0, 4.0, 4.0, 4.0, 5.0, 5.0, 7.0, 9.0])

mean = data.mean()
variance_pop  = data.var()
variance_samp = data.var(ddof=1)
std_pop  = data.std()
std_samp = data.std(ddof=1)

print("Data  :", data)
print(f"Mean               : {mean:.4f}")
print(f"Population variance: {variance_pop:.4f}")
print(f"Sample variance    : {variance_samp:.4f}  (ddof=1, unbiased estimator)")
print(f"Population std     : {std_pop:.4f}")
print(f"Sample std         : {std_samp:.4f}")

manual_variance = ((data - mean) ** 2).mean()
print(f"\nManual variance (showing the formula): {manual_variance:.4f}")
print(f"Deviations from mean: {(data - mean).round(2)}")
print(f"Squared deviations  : {((data - mean)**2).round(2)}")

`ddof=1` means "degrees of freedom = 1". When estimating population variance from a sample, dividing by (n-1) rather than n gives an unbiased estimator — this is Bessel's correction. The intuition: if you only have one sample, you cannot estimate variance at all, so the denominator should be 0, not 1. For large datasets, the difference is negligible, but it matters in small-sample statistics and shows up in PyTorch's `torch.var(x, unbiased=True)` default.

In [ ]:
def layer_norm(x, gamma, beta, eps=1e-5):
    mean = x.mean(axis=-1, keepdims=True)
    var  = x.var(axis=-1, keepdims=True)
    x_hat = (x - mean) / np.sqrt(var + eps)
    return gamma * x_hat + beta

batch_size = 4
d_model    = 8

activations = np.random.randn(batch_size, d_model) * 5 + 3

gamma = np.ones(d_model)
beta  = np.zeros(d_model)

normalized = layer_norm(activations, gamma, beta)

print("Before Layer Norm:")
print(f"  Mean per sample: {activations.mean(axis=1).round(3)}")
print(f"  Std  per sample: {activations.std(axis=1).round(3)}")

print("\nAfter Layer Norm:")
print(f"  Mean per sample: {normalized.mean(axis=1).round(6)}")
print(f"  Std  per sample: {normalized.std(axis=1).round(6)}")

print("\nepsilon (eps=1e-5) prevents division by zero when variance is 0.")

This is a from-scratch implementation of Layer Normalization — one of the two normalization operations you will implement when building the Transformer. The formula is: x_hat = (x - mean) / sqrt(variance + epsilon). After normalization, every feature vector has mean 0 and standard deviation 1. The learnable parameters gamma (scale) and beta (shift) then allow the model to move away from zero mean and unit variance if that is what it needs — they give the normalization step flexibility to undo itself if necessary.

Layer Norm is applied before or after each sub-layer in the Transformer (the exact position depends on the pre-norm vs post-norm architecture variant). Without it, activations drift to very large or very small values across layers, making training unstable.

---
## 4. Information Theory: Entropy, Cross-Entropy, and KL Divergence

### Theory

Information theory, developed by Claude Shannon, quantifies information. The core idea: rare events carry more information than common ones. Seeing a headline "Sun rose this morning" tells you nothing. Seeing "Earthquake in Youngstown" tells you a lot.

**Self-information (surprisal):** The information content of a single event with probability p is: I(x) = -log2(p(x)). If p is close to 1 (certain), I is close to 0. If p is small (rare), I is large.

**Entropy H(P):** The average information (surprise) of a distribution. H(P) = -sum of p(x) * log(p(x)) for all x. Maximum entropy occurs when all outcomes are equally likely (maximum uncertainty). Minimum entropy (zero) occurs when one outcome has probability 1 (complete certainty).

**Cross-Entropy H(P, Q):** The average number of bits needed to encode events from distribution P using a code optimized for distribution Q. H(P, Q) = -sum of p(x) * log(q(x)). When P = Q, cross-entropy equals entropy. When Q is a bad model of P, cross-entropy is larger.

**KL Divergence:** KL(P||Q) = H(P, Q) - H(P). This measures how much extra information is needed because we used Q instead of the true P. It is always >= 0, and equals 0 only when P = Q exactly.

**Where this appears in LLMs:** Cross-entropy loss is the training objective of every language model. The model predicts Q (a distribution over tokens). The true label is P (a one-hot distribution — probability 1 for the correct token, 0 for all others). Minimizing cross-entropy loss means making the model's predictions as close as possible to the true distribution. Perplexity — the standard metric for language models — is 2 raised to the power of cross-entropy (in bits).

In [ ]:
def entropy(probs):
    probs = np.array(probs, dtype=float)
    probs = probs[probs > 0]
    return -np.sum(probs * np.log(probs))

certain     = [1.0, 0.0, 0.0, 0.0]
peaked      = [0.7, 0.1, 0.1, 0.1]
uniform     = [0.25, 0.25, 0.25, 0.25]

print("Entropy measures uncertainty / spread of a distribution")
print(f"  Certain     [1,0,0,0]           : H = {entropy(certain):.4f} nats")
print(f"  Peaked      [0.7,0.1,0.1,0.1]   : H = {entropy(peaked):.4f} nats")
print(f"  Uniform     [0.25,0.25,0.25,0.25]: H = {entropy(uniform):.4f} nats")
print(f"  Max possible for 4 outcomes      : H = {np.log(4):.4f} nats")

n_tokens = [10, 100, 1000, 50257]
print("\nMaximum entropy (log(vocab_size)) for different vocabulary sizes:")
for n in n_tokens:
    print(f"  vocab_size={n:6d}: max H = {np.log(n):.4f} nats  |  perplexity = {n}")

GPT-2 uses a vocabulary of 50,257 tokens (50,000 BPE merges + 256 byte tokens + end-of-text). The maximum entropy for this vocabulary is log(50257) ≈ 10.8 nats, corresponding to a perplexity of 50,257 — what you would get from a completely random model. A well-trained GPT-2 achieves perplexity around 20-40 on standard benchmarks, meaning it is almost as uncertain as if it were choosing between only 20-40 equally likely tokens at each step.

In [ ]:
def cross_entropy(p_true, q_pred):
    p_true = np.array(p_true, dtype=float)
    q_pred = np.array(q_pred, dtype=float)
    q_pred = np.clip(q_pred, 1e-12, 1.0)
    return -np.sum(p_true * np.log(q_pred))

def kl_divergence(p_true, q_pred):
    return cross_entropy(p_true, q_pred) - entropy(p_true)

vocab_size = 5
correct_token = 2

p_true = np.zeros(vocab_size)
p_true[correct_token] = 1.0

q_perfect  = np.array([0.00, 0.00, 1.00, 0.00, 0.00])
q_good     = np.array([0.05, 0.05, 0.80, 0.05, 0.05])
q_mediocre = np.array([0.10, 0.15, 0.40, 0.20, 0.15])
q_bad      = np.array([0.20, 0.20, 0.20, 0.20, 0.20])

print(f"True token: index {correct_token} (one-hot target)")
print(f"\n{'Model':12s}  {'Cross-Entropy':15s}  {'KL Divergence':15s}  {'Perplexity':12s}")
print("-" * 60)

for name, q in [("Perfect", q_perfect), ("Good", q_good),
                ("Mediocre", q_mediocre), ("Random", q_bad)]:
    ce  = cross_entropy(p_true, q)
    kl  = kl_divergence(p_true, q)
    ppl = np.exp(ce)
    print(f"{name:12s}  {ce:15.4f}  {kl:15.4f}  {ppl:12.4f}")

The cross-entropy loss for a one-hot target simplifies to: -log(q[correct_token]). You are just taking the negative log of the probability the model assigned to the correct token. If the model is confident and correct (prob = 0.80), loss = -log(0.80) ≈ 0.22. If the model is uncertain (prob = 0.20), loss = -log(0.20) ≈ 1.61. If the model is confident and wrong (prob = 0.01), loss = -log(0.01) ≈ 4.61. Training pulls the model toward assigning high probability to the correct token.

Perplexity = exp(cross-entropy). A perplexity of 2 means the model is as confused as if it were choosing uniformly between 2 equally likely options at every step. Lower is better.

In [ ]:
def softmax(logits):
    shifted = logits - logits.max()
    exp_l = np.exp(shifted)
    return exp_l / exp_l.sum()

def cross_entropy_loss_batch(logits_batch, targets):
    losses = []
    for logits, target in zip(logits_batch, targets):
        probs = softmax(logits)
        loss  = -np.log(probs[target] + 1e-12)
        losses.append(loss)
    return np.mean(losses)

batch_size = 4
vocab_size = 6

logits  = np.array([[2.3, 0.1, -0.5, 1.2, 0.8, -1.1],
                    [0.5, 3.1, 0.2, -0.3, 0.9, 0.1],
                    [-0.2, 0.8, 2.9, 0.4, -0.6, 1.0],
                    [1.1, -0.5, 0.3, 0.7, 2.4, 0.6]])
targets = np.array([0, 1, 2, 4])

loss = cross_entropy_loss_batch(logits, targets)

print("Batch cross-entropy loss (manual implementation):")
print(f"  Logits shape : {logits.shape}")
print(f"  Targets      : {targets}")
print(f"  Batch loss   : {loss:.4f}")
print(f"  Perplexity   : {np.exp(loss):.4f}")

for i, (lgts, tgt) in enumerate(zip(logits, targets)):
    probs = softmax(lgts)
    print(f"\n  Sample {i}: target={tgt}, "
          f"assigned prob={probs[tgt]:.4f}, "
          f"loss={-np.log(probs[tgt]):.4f}")

This is the complete forward pass of a language model's loss computation. In Raschka's implementation, you will see `torch.nn.functional.cross_entropy(logits, targets)` — this does exactly what is written above: applies softmax internally, picks the probability at the target index, takes the negative log, and averages over the batch. The "numerically stable" softmax subtracts the maximum logit before exponentiation to prevent overflow, which is why the manual implementation includes `logits - logits.max()`.

---
## 5. Maximum Likelihood Estimation

### Theory

Maximum Likelihood Estimation (MLE) is the principle behind training almost every machine learning model, including language models.

**Core idea:** Given observed data, find the model parameters that make the observed data most probable.

If you observe a sequence of tokens [t1, t2, t3, ..., tn], MLE says: find the parameters theta that maximize P(t1, t2, ..., tn | theta).

By the chain rule: P(t1, ..., tn | theta) = P(t1|theta) * P(t2|t1, theta) * ... * P(tn|t1,...,t_{n-1}, theta)

Taking the log (log turns products into sums, and does not change where the maximum is):
log P = sum of log P(t_i | t_1, ..., t_{i-1}, theta)

Maximizing log P is identical to minimizing negative log P — the cross-entropy loss.

**Key insight:** Training a language model with cross-entropy loss is exactly MLE. You are finding model parameters that maximize the probability of the training data. This is not a heuristic — it has a rigorous statistical justification.

**Where this appears in LLMs:** Every gradient descent step during LLM pretraining is a step toward the MLE solution. The training objective written in every paper as "minimize cross-entropy loss" is mathematically identical to "maximize the likelihood of the training corpus."

In [ ]:
observed_heads = 7
n_flips        = 10

theta_candidates = np.linspace(0.01, 0.99, 200)

from math import comb
likelihoods = np.array([
    comb(n_flips, observed_heads) * (t ** observed_heads) * ((1-t) ** (n_flips - observed_heads))
    for t in theta_candidates
])
log_likelihoods = np.log(likelihoods + 1e-300)

mle_theta = theta_candidates[likelihoods.argmax()]
analytical_mle = observed_heads / n_flips

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(theta_candidates, likelihoods, 'steelblue', linewidth=2)
plt.axvline(mle_theta, color='red', linestyle='--', label=f'MLE theta = {mle_theta:.2f}')
plt.xlabel('theta (coin bias)')
plt.ylabel('Likelihood')
plt.title('Likelihood vs theta')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(theta_candidates, -log_likelihoods, 'coral', linewidth=2)
plt.axvline(mle_theta, color='red', linestyle='--', label=f'Minimum at {mle_theta:.2f}')
plt.xlabel('theta (coin bias)')
plt.ylabel('Negative Log-Likelihood (= Loss)')
plt.title('Negative Log-Likelihood (the loss we minimize)')
plt.legend()

plt.tight_layout()
plt.show()

print(f"Observed: {observed_heads} heads in {n_flips} flips")
print(f"MLE estimate (numerical)  : theta = {mle_theta:.4f}")
print(f"MLE estimate (analytical) : theta = {analytical_mle:.4f}")
print(f"Intuition: the best estimate is the empirical frequency.")

The left plot shows the likelihood — higher is better. The right plot shows the negative log-likelihood — lower is better. They have their optimal point at the same theta, but we minimize the negative log-likelihood because gradient descent minimizes (not maximizes) and because logarithms turn products into sums which are numerically more stable.

For a language model, theta represents all model weights (hundreds of millions of parameters), and the likelihood is the probability the model assigns to the entire training corpus. Gradient descent on the cross-entropy loss is MLE via stochastic approximation.

In [ ]:
training_tokens = ['the', 'cat', 'sat', 'on', 'the', 'mat', 'the', 'cat', 'ran', 'away']

from collections import Counter

unigram_counts = Counter(training_tokens)
total = len(training_tokens)
unigram_probs  = {token: count / total for token, count in unigram_counts.items()}

bigram_counts = Counter(zip(training_tokens[:-1], training_tokens[1:]))

bigram_probs = {}
for (w1, w2), count in bigram_counts.items():
    bigram_probs[(w1, w2)] = count / unigram_counts[w1]

print("Unigram MLE probabilities:")
for token, prob in sorted(unigram_probs.items(), key=lambda x: -x[1]):
    print(f"  P('{token}') = {prob:.4f}")

print("\nBigram MLE probabilities (P(w2 | w1)):")
for (w1, w2), prob in sorted(bigram_probs.items(), key=lambda x: -x[1]):
    print(f"  P('{w2}' | '{w1}') = {prob:.4f}")

test_sequence = ['the', 'cat', 'ran']
log_prob = 0.0
log_prob += np.log(unigram_probs[test_sequence[0]])
for i in range(1, len(test_sequence)):
    pair = (test_sequence[i-1], test_sequence[i])
    p = bigram_probs.get(pair, 1e-10)
    log_prob += np.log(p)

print(f"\nLog P('{' '.join(test_sequence)}') = {log_prob:.4f}")
print(f"Probability                      = {np.exp(log_prob):.6f}")

This is a bigram language model estimated by MLE. The estimated probability of each token given the previous token is simply the relative frequency in the training data — the MLE solution for this model class. A neural language model (GPT) generalizes this to condition on the entire context, not just the previous token, and the MLE estimation is done via gradient descent rather than direct counting. The counting approach fails for large vocabularies and contexts because most n-grams never appear in training data.

---
## 6. Bayes' Theorem and Conditional Probability

### Theory

Bayes' theorem is one of the most important results in probability theory. It tells you how to update your beliefs when you receive new evidence.

**Conditional probability:** P(A|B) is the probability of A given that B has already occurred. P(A|B) = P(A and B) / P(B).

**Bayes' theorem:** P(A|B) = P(B|A) * P(A) / P(B)

In machine learning language:
- P(theta | data) is the posterior — what we believe about parameters after seeing data
- P(data | theta) is the likelihood — how probable the data is given these parameters
- P(theta) is the prior — what we believed about parameters before seeing data
- P(data) is the evidence — a normalizing constant

**Where this appears in LLMs:** The language model itself computes P(next token | context) — this is a conditional probability. Bayesian interpretations of regularization: L2 regularization is equivalent to placing a Gaussian prior on weights. Understanding that temperature scaling during generation is related to Bayesian posterior sharpening.

In [ ]:
p_spam         = 0.30
p_ham          = 0.70
p_word_spam    = 0.80
p_word_ham     = 0.05

p_word = p_word_spam * p_spam + p_word_ham * p_ham

p_spam_given_word = (p_word_spam * p_spam) / p_word
p_ham_given_word  = (p_word_ham  * p_ham)  / p_word

print("Bayesian Text Classification")
print(f"\nPrior: P(spam) = {p_spam}, P(ham) = {p_ham}")
print(f"Likelihood: P('free' | spam) = {p_word_spam}, P('free' | ham) = {p_word_ham}")
print(f"Evidence:   P('free') = {p_word:.4f}")
print(f"\nPosterior after seeing 'free':")
print(f"  P(spam | 'free') = {p_spam_given_word:.4f}")
print(f"  P(ham  | 'free') = {p_ham_given_word:.4f}")
print(f"  Sum = {p_spam_given_word + p_ham_given_word:.4f}")

print("\nPrior:     spam was 30% likely")
print(f"Posterior: spam is now {p_spam_given_word*100:.1f}% likely after seeing the word 'free'")

Bayes' theorem tells you how to update probabilities with new information. A language model does this at every token — it starts with knowledge from pretraining (the prior) and updates based on the context it has seen (the evidence) to produce a posterior distribution over the next token. This is why GPT produces sensible continuations: context shifts the probability distribution dramatically, ruling out tokens that are incompatible with what came before.

In [ ]:
np.random.seed(0)

true_weight = 2.5
n_points    = 20
X = np.random.randn(n_points)
y = true_weight * X + np.random.randn(n_points) * 1.0

w_mle = (X * y).sum() / (X * X).sum()

lambda_reg = 0.5
w_map = (X * y).sum() / (X * X).sum() + lambda_reg)

w_map = (X * y).sum() / ((X * X).sum() + lambda_reg)

print("L2 Regularization as a Bayesian Prior")
print(f"True weight    : {true_weight}")
print(f"MLE estimate   : {w_mle:.4f}  (maximizes likelihood, no prior)")
print(f"MAP estimate   : {w_map:.4f}  (maximizes posterior, Gaussian prior on w)")
print(f"\nThe MAP estimate is 'pulled' toward 0 by the prior.")
print(f"L2 regularization strength lambda = {lambda_reg}")
print(f"Larger lambda → stronger pull toward 0 → smaller weights → less overfitting")

Maximum A Posteriori (MAP) estimation is MLE with a prior. Adding L2 regularization (weight decay) to a neural network is mathematically equivalent to placing a Gaussian prior on the weights — the prior encodes the belief that weights should be small. This is why weight decay in AdamW (the optimizer used to train GPT models) is not just a heuristic trick but has a principled Bayesian interpretation. It prevents the model from fitting noise in the training data.

---
## 7. The Normal Distribution and Why It Matters

### Theory

The Normal (Gaussian) distribution is described by two parameters: mean (mu) and variance (sigma^2). Its probability density function is:

f(x) = (1 / sqrt(2 * pi * sigma^2)) * exp(-(x - mu)^2 / (2 * sigma^2))

**The 68-95-99.7 rule:** 68% of data falls within 1 standard deviation of the mean, 95% within 2, and 99.7% within 3.

**Standard normal:** mu = 0, sigma = 1. Normalizing data (subtracting mean, dividing by std) produces the standard normal distribution.

**Why the normal distribution is everywhere:**
The Central Limit Theorem (section 11) explains this — sums of many independent random variables converge to a normal distribution regardless of the original distribution. Gradients in a neural network are sums of many small contributions, so they tend toward normality.

**Where this appears in LLMs:** Weight initialization (normal with small std). Layer normalization explicitly maps activations to zero mean and unit variance. Gradient noise during training. The noise added in diffusion models follows a normal distribution. Understanding when distributions deviate from normal is important for debugging training instabilities.

In [ ]:
x = np.linspace(-5, 5, 1000)

configs = [(0, 1, 'blue',  'mu=0, sigma=1  (standard normal)'),
           (0, 2, 'red',   'mu=0, sigma=2  (wider)'),
           (2, 1, 'green', 'mu=2, sigma=1  (shifted right)')]

plt.figure(figsize=(11, 4))
for mu, sigma, color, label in configs:
    pdf = stats.norm.pdf(x, mu, sigma)
    plt.plot(x, pdf, color=color, linewidth=2, label=label)

standard_pdf = stats.norm.pdf(x, 0, 1)
plt.fill_between(x, standard_pdf,
                 where=(x >= -1) & (x <= 1),
                 alpha=0.15, color='blue', label='68% within 1 sigma')
plt.fill_between(x, standard_pdf,
                 where=(x >= -2) & (x <= 2),
                 alpha=0.08, color='blue', label='95% within 2 sigma')

plt.xlabel('Value')
plt.ylabel('Probability Density')
plt.title('Normal Distributions')
plt.legend()
plt.show()

samples_std01 = np.random.normal(0, 0.02, 10000)
samples_std1  = np.random.normal(0, 1.0,  10000)

print("GPT-2 weight init (std=0.02):")
print(f"  % within ±0.04 (2-sigma): {(np.abs(samples_std01) <= 0.04).mean()*100:.1f}%")
print(f"  Max absolute value       : {np.abs(samples_std01).max():.4f}")

print("\nStandard normal (std=1.0):")
print(f"  % within ±2 (2-sigma)   : {(np.abs(samples_std1) <= 2.0).mean()*100:.1f}%")

GPT-2 initializes weights from N(0, 0.02). This means 95% of weights are between -0.04 and +0.04. These very small values are intentional — they keep the initial activations in a reasonable range before training begins. If weights were drawn from N(0, 1), initial activations in a 768-dimensional model would have standard deviation around sqrt(768) ≈ 27, which would push most inputs into the saturated regions of activation functions, causing near-zero gradients and very slow learning.

---
## 8. Sampling and Temperature

### Theory

When generating text, a language model produces a probability distribution over the next token. There are several strategies for choosing a token from this distribution:

**Greedy decoding:** Always pick the highest probability token. Deterministic, but often repetitive and uninteresting.

**Random sampling:** Sample from the distribution as-is. Can produce incoherent text when many low-probability tokens exist.

**Temperature scaling:** Divide the logits by a temperature T before applying softmax. T > 1 makes the distribution flatter (more random). T < 1 makes it sharper (more confident). T = 1 is the default. T → 0 approaches greedy decoding.

**Top-k sampling:** Keep only the k highest-probability tokens, redistribute probability among them, and sample.

**Top-p (nucleus) sampling:** Keep the smallest set of tokens whose cumulative probability exceeds p. This adapts to the shape of the distribution automatically.

**Where this appears in LLMs:** The `generate()` function in every language model implementation uses these strategies. Raschka implements temperature sampling in the generation code. Understanding this connects probability theory to the actual text you see a language model produce.

In [ ]:
def softmax_with_temperature(logits, temperature=1.0):
    scaled = logits / temperature
    shifted = scaled - scaled.max()
    exp_l = np.exp(shifted)
    return exp_l / exp_l.sum()

vocab_sample = ['the', 'a', 'an', 'one', 'this', 'that', 'some', 'every']
logits = np.array([3.2, 2.1, 1.5, 0.8, 1.2, 0.9, 0.6, 0.3])

temperatures = [0.1, 0.5, 1.0, 2.0, 5.0]

fig, axes = plt.subplots(1, len(temperatures), figsize=(16, 4), sharey=False)

for ax, T in zip(axes, temperatures):
    probs = softmax_with_temperature(logits, T)
    ax.bar(range(len(vocab_sample)), probs, color='steelblue', alpha=0.8)
    ax.set_title(f'T = {T}')
    ax.set_xticks(range(len(vocab_sample)))
    ax.set_xticklabels(vocab_sample, rotation=45, ha='right', fontsize=8)
    ax.set_ylabel('Probability')
    entropy_val = -np.sum(probs * np.log(probs + 1e-12))
    ax.set_xlabel(f'H = {entropy_val:.2f}')

plt.suptitle('Effect of Temperature on Token Distribution', fontsize=12)
plt.tight_layout()
plt.show()

print("Temperature effect on distribution entropy:")
for T in temperatures:
    probs = softmax_with_temperature(logits, T)
    H = -np.sum(probs * np.log(probs + 1e-12))
    print(f"  T={T:4.1f} | entropy={H:.4f} | max_prob={probs.max():.4f} | '{vocab_sample[probs.argmax()]}' ")

At T=0.1, the distribution is extremely peaked — the model almost certainly produces 'the' every time. Output is predictable and repetitive. At T=5.0, the distribution is nearly uniform — the model samples almost randomly. Output is creative but often incoherent. T=1.0 is the model's natural uncertainty. T between 0.7 and 1.2 is the typical range used in practice for coherent but varied generation. This is why ChatGPT has different creative writing modes vs precise factual modes — they correspond to different temperatures.

In [ ]:
def top_k_sampling(logits, k, temperature=1.0):
    top_k_indices = np.argsort(logits)[-k:]
    top_k_logits  = logits[top_k_indices]
    probs = softmax_with_temperature(top_k_logits, temperature)
    chosen_idx = np.random.choice(len(probs), p=probs)
    return top_k_indices[chosen_idx]

def top_p_sampling(logits, p, temperature=1.0):
    probs = softmax_with_temperature(logits, temperature)
    sorted_idx  = np.argsort(probs)[::-1]
    sorted_probs = probs[sorted_idx]
    cumsum = np.cumsum(sorted_probs)
    cutoff = np.searchsorted(cumsum, p) + 1
    nucleus_idx   = sorted_idx[:cutoff]
    nucleus_probs = probs[nucleus_idx]
    nucleus_probs = nucleus_probs / nucleus_probs.sum()
    chosen = np.random.choice(nucleus_idx, p=nucleus_probs)
    return chosen

np.random.seed(123)
logits = np.array([3.2, 2.1, 1.5, 0.8, 1.2, 0.9, 0.6, 0.3])

print(f"Vocabulary: {vocab_sample}")
print(f"Base probs: {softmax_with_temperature(logits, 1.0).round(3)}")

print("\nTop-k=3 samples (10 draws):")
samples = [vocab_sample[top_k_sampling(logits, k=3)] for _ in range(10)]
print(" ", samples)

print("\nTop-p=0.9 samples (10 draws):")
samples = [vocab_sample[top_p_sampling(logits, p=0.9)] for _ in range(10)]
print(" ", samples)

probs = softmax_with_temperature(logits, 1.0)
sorted_probs = np.sort(probs)[::-1]
cumsum = np.cumsum(sorted_probs)
print(f"\nCumulative probability (sorted desc): {cumsum.round(3)}")
print(f"Top-p=0.90 uses the first {np.searchsorted(cumsum, 0.90)+1} tokens")

Top-p (nucleus) sampling is more adaptive than top-k. When the model is very confident, only a few tokens will have high probability, and the nucleus will be small. When the model is uncertain, the nucleus will include more tokens. This is generally preferred over top-k in practice. GPT-2 and later models use top-p sampling with p=0.9 to 0.95 as the default generation strategy.

---
## 9. Descriptive Statistics for Model Monitoring

### Theory

During training, you constantly monitor statistical properties of your model to detect problems. Understanding what these statistics mean and what healthy vs unhealthy training looks like is a practical skill.

**Key quantities to monitor:**
- Loss value and its rate of decrease
- Gradient norms (if exploding → training is unstable; if vanishing → model not learning)
- Weight norms (growing weights indicate insufficient regularization)
- Activation statistics (mean and variance through layers)
- Learning rate (scheduled over training)

**Gradient clipping:** When gradient norms exceed a threshold, scale the entire gradient vector down so its norm equals the threshold. This prevents individual large gradients from destabilizing training. Used in every large model training run.

**Where this appears in LLMs:** Every training script logs loss, gradient norm, and learning rate. Raschka's training loop computes these. Understanding them helps you know when to stop training, adjust hyperparameters, or diagnose bugs.

In [ ]:
np.random.seed(0)

n_steps = 200
t = np.arange(n_steps)

train_loss = 4.0 * np.exp(-t / 60) + 0.8 + np.random.randn(n_steps) * 0.15
val_loss   = 4.0 * np.exp(-t / 60) + 1.0 + np.random.randn(n_steps) * 0.12

val_loss[120:] += np.linspace(0, 0.8, 80)

gradient_norms = 2.0 * np.exp(-t / 40) + 0.3 + np.abs(np.random.randn(n_steps) * 0.2)
gradient_norms[50:70] += np.random.exponential(2.0, 20)

window = 10
train_smooth = np.convolve(train_loss, np.ones(window)/window, mode='valid')
val_smooth   = np.convolve(val_loss, np.ones(window)/window, mode='valid')

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(train_loss, alpha=0.3, color='steelblue')
axes[0].plot(np.arange(window-1, n_steps), train_smooth, color='steelblue', label='Train loss')
axes[0].plot(val_loss, alpha=0.3, color='coral')
axes[0].plot(np.arange(window-1, n_steps), val_smooth, color='coral', label='Val loss')
axes[0].axvline(120, color='black', linestyle='--', alpha=0.5, label='Overfitting begins')
axes[0].set_xlabel('Training step')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()

clip_thresh = 1.0
axes[1].plot(gradient_norms, color='mediumseagreen', alpha=0.7, label='Gradient norm')
axes[1].axhline(clip_thresh, color='red', linestyle='--', label=f'Clip threshold = {clip_thresh}')
clipped = np.minimum(gradient_norms, clip_thresh)
axes[1].plot(clipped, color='darkgreen', linewidth=1.5, label='After clipping')
axes[1].set_xlabel('Training step')
axes[1].set_ylabel('Gradient L2 norm')
axes[1].set_title('Gradient Norms and Clipping')
axes[1].legend()

plt.tight_layout()
plt.show()

print("Training statistics summary:")
print(f"  Final train loss     : {train_loss[-10:].mean():.4f} +/- {train_loss[-10:].std():.4f}")
print(f"  Final val loss       : {val_loss[-10:].mean():.4f} +/- {val_loss[-10:].std():.4f}")
print(f"  Overfitting gap      : {(val_loss[-10:] - train_loss[-10:]).mean():.4f}")
print(f"  Gradient spike steps : {(gradient_norms > 2.0).sum()} steps exceeded 2.0")
print(f"  % of gradients clipped: {(gradient_norms > clip_thresh).mean()*100:.1f}%")

The left plot shows a classic overfitting signature: training loss continues to decrease after step 120, but validation loss starts increasing. This is the point to stop training (early stopping) or increase regularization. The right plot shows gradient spikes between steps 50-70 — these could destabilize training without gradient clipping. PyTorch's `torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)` implements the clipping shown here.

---
## 10. Correlation and Covariance

### Theory

**Covariance:** Measures how two variables change together. Cov(X, Y) = E[(X - E[X])(Y - E[Y])]. Positive covariance: when X is above its mean, Y tends to be above its mean. Negative covariance: they move in opposite directions. Zero covariance: no linear relationship.

**Correlation:** Normalized covariance. Corr(X, Y) = Cov(X, Y) / (SD(X) * SD(Y)). Ranges from -1 to +1. This removes the effect of scale.

**Covariance matrix:** For a dataset with d features, the d x d covariance matrix Sigma has Sigma[i,j] = Cov(feature_i, feature_j). Diagonal entries are variances. Off-diagonal entries measure relationships between features.

**Where this appears in LLMs:** Attention mechanisms compute a form of similarity (related to correlation) between token representations. The attention score between query q and key k is q.T @ k — this is a dot product that measures alignment between vectors, closely related to covariance when the vectors are zero-centered. Understanding covariance matrices is important for understanding PCA, which is used to visualize learned embeddings.

In [ ]:
np.random.seed(42)
n = 200

x = np.random.randn(n)
y_positive = 0.8 * x + 0.6 * np.random.randn(n)
y_negative = -0.8 * x + 0.6 * np.random.randn(n)
y_none     = np.random.randn(n)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, y, title in [
    (axes[0], y_positive, 'Positive correlation'),
    (axes[1], y_negative, 'Negative correlation'),
    (axes[2], y_none,     'No correlation')
]:
    r = np.corrcoef(x, y)[0, 1]
    ax.scatter(x, y, alpha=0.4, s=20, color='steelblue')
    ax.set_title(f'{title}\nr = {r:.3f}')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

plt.tight_layout()
plt.show()

embedding_dim = 4
n_tokens = 100
embeddings = np.random.randn(n_tokens, embedding_dim)

cov_matrix = np.cov(embeddings.T)
corr_matrix = np.corrcoef(embeddings.T)

print("Covariance matrix of token embeddings (4D):")
print(cov_matrix.round(3))
print("\nCorrelation matrix:")
print(corr_matrix.round(3))

The correlation matrix of token embeddings tells you how aligned the different embedding dimensions are. Well-trained embeddings have low correlation between dimensions — each dimension captures something independent. High correlation between dimensions means redundancy — the model is wasting capacity. This is why whitening (decorrelating) embeddings is sometimes applied as a post-processing step.

In [ ]:
def scaled_dot_product_attention(Q, K, V):
    d_k     = Q.shape[-1]
    scores  = Q @ K.T / np.sqrt(d_k)
    exp_s   = np.exp(scores - scores.max(axis=-1, keepdims=True))
    weights = exp_s / exp_s.sum(axis=-1, keepdims=True)
    output  = weights @ V
    return output, weights

np.random.seed(1)
seq_len = 5
d_k     = 4

Q = np.random.randn(seq_len, d_k)
K = np.random.randn(seq_len, d_k)
V = np.random.randn(seq_len, d_k)

output, attn_weights = scaled_dot_product_attention(Q, K, V)

print("Scaled Dot-Product Attention")
print(f"Q shape: {Q.shape}, K shape: {K.shape}, V shape: {V.shape}")
print(f"Output shape: {output.shape}")
print(f"\nAttention weight matrix (rows=queries, cols=keys):")
print(attn_weights.round(3))
print(f"\nEach row sums to: {attn_weights.sum(axis=1).round(4)}")
print(f"\nThe scores Q @ K.T measure cosine-like similarity.")
print(f"High score = query and key are aligned = high attention weight.")

The attention score between query vector q_i and key vector k_j is their dot product: q_i · k_j. This measures the cosine similarity (up to scaling by the vector norms) — how aligned the two vectors are in the embedding space. Tokens with high alignment get high attention weights and contribute more to the output. The 1/sqrt(d_k) scaling prevents the dot products from becoming too large in high dimensions, which would push the softmax into regions where gradients vanish. This statistical intuition — dot product as similarity — is the core mathematical idea behind all attention mechanisms.

---
## 11. The Central Limit Theorem

### Theory

The Central Limit Theorem (CLT) is one of the most profound results in statistics.

**Statement:** If you take many independent random variables from any distribution with finite mean mu and variance sigma^2, their sum (or average) will be approximately normally distributed, regardless of the shape of the original distribution. The approximation improves as you take more variables.

Formally: if X_1, X_2, ..., X_n are i.i.d. with mean mu and variance sigma^2, then as n → infinity:
sqrt(n) * (X_bar - mu) / sigma → N(0, 1)

**Why this matters in deep learning:**
A neural network activation is a sum of many weighted inputs. By CLT, even if the weights and inputs are not normally distributed, their weighted sum tends toward a normal distribution. This is why the normality assumption in Layer Norm and Batch Norm is not unreasonable. It is also why gradient distributions tend toward normality, and why normal-based initialization schemes work well in practice.

In [ ]:
np.random.seed(0)

n_experiments = 5000

fig, axes = plt.subplots(2, 4, figsize=(16, 7))

distributions = [
    ('Uniform', lambda n: np.random.uniform(0, 1, n)),
    ('Exponential', lambda n: np.random.exponential(1, n)),
]

sample_sizes = [1, 2, 10, 100]

for row, (dist_name, dist_fn) in enumerate(distributions):
    for col, n in enumerate(sample_sizes):
        sample_means = [dist_fn(n).mean() for _ in range(n_experiments)]
        sample_means = np.array(sample_means)

        axes[row, col].hist(sample_means, bins=50, density=True, color='steelblue', alpha=0.7)

        mu    = sample_means.mean()
        sigma = sample_means.std()
        x_range = np.linspace(mu - 4*sigma, mu + 4*sigma, 200)
        axes[row, col].plot(x_range, stats.norm.pdf(x_range, mu, sigma), 'r-', linewidth=2)

        axes[row, col].set_title(f'{dist_name}\nn = {n}')
        axes[row, col].set_xlabel('Sample mean')
        if col == 0:
            axes[row, col].set_ylabel('Density')

plt.suptitle('Central Limit Theorem: Sample means approach Normal as n increases', fontsize=11)
plt.tight_layout()
plt.show()

The top row starts from a uniform distribution, the bottom from an exponential (highly skewed). At n=1, they look like their original shapes. At n=10, both look approximately bell-shaped. At n=100, both are almost perfectly normal. The red line is the theoretical normal distribution — notice how closely the histograms match it.

In a neural network, each neuron computes a weighted sum of hundreds of inputs. The CLT tells us this sum will be approximately normally distributed even if each input is not. This is why Gaussian assumptions about activations are not wild — they are grounded in CLT. It also means that strategies developed for normal distributions (like Layer Norm) have a principled basis for working in practice.

In [ ]:
np.random.seed(0)

n_in   = 768
n_samples = 5000

inputs  = np.random.randn(n_samples, n_in)
weights = np.random.normal(0, 0.02, (n_in, 1))

activations = (inputs @ weights).flatten()

stat, p_value = stats.normaltest(activations)

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].hist(inputs[:, 0], bins=60, density=True, color='coral', alpha=0.7, label='Single input')
x = np.linspace(-4, 4, 300)
axes[0].plot(x, stats.norm.pdf(x), 'k-', linewidth=2)
axes[0].set_title('Distribution of a single input')
axes[0].legend()

axes[1].hist(activations, bins=60, density=True, color='steelblue', alpha=0.7, label='Weighted sum (activation)')
mu, sigma = activations.mean(), activations.std()
x_r = np.linspace(mu-4*sigma, mu+4*sigma, 300)
axes[1].plot(x_r, stats.norm.pdf(x_r, mu, sigma), 'r-', linewidth=2, label='Normal fit')
axes[1].set_title('Distribution of activation (weighted sum of 768 inputs)')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Activation stats: mean={activations.mean():.6f}, std={activations.std():.6f}")
print(f"Normality test p-value: {p_value:.4f}  (high p → consistent with normality)")

The activation is a weighted sum of 768 inputs. Even though individual inputs are normally distributed here, the key point generalizes: even non-normal inputs produce approximately normal activations in high-dimensional layers, by the CLT. The mean of the activation is near zero (because weights are zero-mean) and the variance is small (because weights have std=0.02). This is why weight initialization with small standard deviation keeps activations in a stable range at the start of training.

---
## 12. Putting It All Together: A Language Model Probability Chain

This final section shows how every concept in this notebook connects in a single generation step of a language model.

In [ ]:
np.random.seed(7)

vocab = ['the', 'a', 'cat', 'dog', 'sat', 'ran', 'on', 'by', 'mat', 'rug',
         'quickly', 'slowly', 'and', 'or', 'but', '<EOS>']
vocab_size = len(vocab)
d_model    = 8

context    = ['the', 'cat']
context_idx = [vocab.index(w) for w in context]

embeddings = np.random.normal(0, 0.02, (vocab_size, d_model))

context_vectors = embeddings[context_idx]

pos_encoding = np.array([[np.sin(pos / (10000 ** (2*i/d_model)))
                          if i % 2 == 0
                          else np.cos(pos / (10000 ** (2*(i//2)/d_model)))
                          for i in range(d_model)]
                         for pos in range(len(context))])

context_vectors = context_vectors + pos_encoding

mean_cv = context_vectors.mean(axis=-1, keepdims=True)
std_cv  = context_vectors.std(axis=-1, keepdims=True)
norm_cv = (context_vectors - mean_cv) / (std_cv + 1e-5)

W_q = np.random.normal(0, 0.02, (d_model, d_model))
W_k = np.random.normal(0, 0.02, (d_model, d_model))
W_v = np.random.normal(0, 0.02, (d_model, d_model))

Q = norm_cv @ W_q
K = norm_cv @ W_k
V = norm_cv @ W_v

d_k = d_model
scores = Q @ K.T / np.sqrt(d_k)
exp_scores  = np.exp(scores - scores.max(axis=-1, keepdims=True))
attn_weights = exp_scores / exp_scores.sum(axis=-1, keepdims=True)
attended = attn_weights @ V

W_out = np.random.normal(0, 0.02, (d_model, vocab_size))
logits = attended[-1] @ W_out

temperature    = 0.8
logits_scaled  = logits / temperature
logits_shifted = logits_scaled - logits_scaled.max()
probs = np.exp(logits_shifted) / np.exp(logits_shifted).sum()

H = -np.sum(probs * np.log(probs + 1e-12))

chosen_idx   = np.random.choice(vocab_size, p=probs)
chosen_token = vocab[chosen_idx]

print("Complete single generation step:")
print(f"  Context          : {context}")
print(f"  Embedding dim    : {d_model}")
print(f"  Vocab size       : {vocab_size}")
print(f"  Attention weights: {attn_weights.round(3)}")
print(f"  Temperature      : {temperature}")
print(f"  Output entropy   : {H:.4f} nats")
print(f"  Perplexity       : {np.exp(H):.4f}")
print(f"\n  Token probabilities:")
sorted_idx = np.argsort(probs)[::-1]
for i in sorted_idx[:6]:
    print(f"    P('{vocab[i]}') = {probs[i]:.4f}")
print(f"\n  Sampled next token: '{chosen_token}'")
print(f"  Negative log prob (loss): {-np.log(probs[chosen_idx]):.4f}")

Every step in this code connects to a concept from this notebook:

- **Embeddings initialized with normal distribution** (Section 7): small std prevents activation explosion
- **Positional encoding** uses sinusoidal functions — deterministic, not learned
- **Layer normalization** (Section 3): maps each vector to zero mean, unit variance
- **Attention scores = dot product / sqrt(d_k)** (Section 10): dot product measures correlation/alignment between query and key
- **Softmax on scores** (Section 1): converts scores to a probability distribution over positions
- **Logits** are raw scores before normalization
- **Temperature scaling** (Section 8): controls sharpness of the output distribution
- **Entropy of output** (Section 4): measures model uncertainty at this step
- **Sampling from the categorical distribution** (Section 2): selects the next token
- **Negative log probability = cross-entropy loss** (Section 4): what would be minimized during training

This is the complete picture. Every mathematical operation in an LLM is a direct application of the statistical concepts in this notebook. When you read Raschka's implementation, you now have the statistical vocabulary to understand not just what each line does, but why it was designed that way.

---
## Reference Summary

| Concept | Formula | Where in LLMs |
|---|---|---|
| Probability | P(A) in [0,1], sums to 1 | Every model output is a distribution |
| Chain rule | P(A,B) = P(A) * P(B\|A) | Language modeling objective |
| Entropy | H = -sum p * log(p) | Measures uncertainty; equals loss at optimal model |
| Cross-entropy | H(P,Q) = -sum p * log(q) | Training loss function |
| KL divergence | KL = H(P,Q) - H(P) | Fine-tuning objectives (RLHF uses KL penalty) |
| Perplexity | PPL = exp(H) | Standard LLM evaluation metric |
| MLE | argmax_theta P(data\|theta) | Training = minimizing cross-entropy = MLE |
| MAP / L2 reg | MLE + Gaussian prior | Weight decay in AdamW |
| Mean | (1/n) sum x_i | Layer Norm computation |
| Variance | E[(x-mu)^2] | Layer Norm computation |
| Layer Norm | (x - mu) / sigma | Stabilizes activations in every Transformer block |
| Normal dist | N(mu, sigma^2) | Weight initialization (GPT-2: N(0, 0.02)) |
| Softmax | exp(x_i) / sum exp(x_j) | Converts logits to probabilities |
| Temperature | softmax(logits / T) | Controls generation randomness |
| Top-k / Top-p | Truncate low-prob tokens | Generation strategies |
| Dot product | q . k = sum q_i * k_i | Attention score = similarity between tokens |
| Gradient norm | ||grad||_2 | Gradient clipping threshold |
| CLT | sums → Normal | Why Gaussian assumptions hold for activations |
| Covariance | E[(x-mu_x)(y-mu_y)] | Embedding alignment, attention patterns |